# Validation of peak Active Power values for 20 test grids

data / ls-meten / ls_meten_20_e1_testgrids / **{msr_id}** / **{msr_id}** + "-EL-L01-0T1"  / ActivePower / ABC / Average / {first file in folder}

In [1]:
import pickle
import os
from pathlib import Path

import pandas as pd

#### Utility functions for loading LS-meetdata

In [2]:
BASE_PATH = Path("../custom-file-systems/efs/fs-01aa7e2484bb89cc8/ls-meten/ls_meten_20_e1_testgrids")

In [3]:
def create_path_to_pickle_file(msr_id: str):
    # Return the filepath to the trafo measurement data of the msr.
    base_path = BASE_PATH
    msr_path = base_path / msr_id

    for dir in msr_path.rglob("*"):
        if dir.name.endswith("0T1"):
            folder_name = dir.name
            break

    target_folder = base_path / msr_id / folder_name / "ActivePower" / "ABC" / "Average"

    # Get the first file in the target folder
    first_file = next(target_folder.glob('*'), None)

    if first_file is None:
        raise FileNotFoundError("No files found in the specified folder.")

    return first_file

In [4]:
def read_pickle_file(file_path: Path) -> pd.DataFrame:
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

In [6]:
msr_ids = [
    "1002145", "1002492", "1002991", "2000177", "2001773",
    "2001981", "3000483", "3017983", "3021828", "4003456",
    "4003698", "4006111", "5001314", "6001956", "6003261",
    "6003268", "6003385", "6003615", "6004634", "7003817",
]

peak_moments = []
peaks_active_power = []

for msr_id in msr_ids:
    path = create_path_to_pickle_file(msr_id)
    data = read_pickle_file(path)
    row_index_peak_ldn = data.idxmin()["measurement_value"]
    peak_moments.append(data.iloc[row_index_peak_ldn]["timestamp"])
    peaks_active_power.append(data.iloc[row_index_peak_ldn]["measurement_value"])

peak_df = pd.DataFrame(data=[msr_ids, peak_moments, peaks_active_power]).T
peak_df.columns = ["msr_id", "peak_timestamp", "peak_active_power"]
peak_df.peak_active_power = peak_df.peak_active_power * -1
peak_df.head(20)

,msr_id,peak_timestamp,peak_active_power
0,1002145,2024-01-09 17:25:00+00:00,208.173996
1,1002492,2023-12-03 18:25:00+00:00,169.065994
2,1002991,2023-12-05 17:25:00+00:00,190.442001
3,2000177,2023-12-03 16:50:00+00:00,289.618988
4,2001773,2024-01-17 16:55:00+00:00,282.502991
5,2001981,2024-01-08 17:05:00+00:00,345.507996
6,3000483,2024-01-09 16:30:00+00:00,234.690994
7,3017983,2024-01-08 17:20:00+00:00,264.325012
8,3021828,2024-01-14 16:50:00+00:00,202.501999
9,4003456,2023-11-29 16:15:00+00:00,235.781006
